## Домашняя работа к Уроку 2

Написать приложение и функцию, которые собирают основные новости с сайта на выбор lenta.ru, mail.ru, dzen.ru . Для парсинга использовать XPath
Структура данных должна содержать:
* название источника
* наименование новости
* ссылку на новость
* дата публикации
минимум один сайт максимум все.

Подсказка 1. У ленты нет даты в новости. Посмотрите откуда ее взять
Подсказка 2. У дзена и mail нет своих новостей. Нужно найти источник.

Домашнее задание нужно приложить как ссылка на репозиторий. Для этого вставьте ссылку на репозиторий в текстовое окно и нажмите кнопку "сдать".
Задание считается выполненным, если:
- код задокументирован, парсер работает корректно
- репозиторий имеет readme.md с описанием и структурой проекта
- есть скриншоты результата сбора данных и обращения к БД

### Импорт необходимых библиотек

In [1]:
import requests
from lxml import html
from datetime import date
from datetime import datetime
import json

### Функция сбора новостей

#### Определим url новостей

In [2]:
# Создадим список url новостных сайтов
news_site_urls_dict = {
    'lenta': 'https://lenta.ru/parts/news/', 
    'mail': 'https://news.mail.ru/', 
    'dzen': 'https://dzen.ru/'
}

# Создадим словари путей xpath

# идентификатор пути xpath с заголовком новости
xpath_news_name_dict = {
    'lenta': "//div/ul/li/a/h3[@class='card-full-news__title']/text()",
    # На сайте mail.ru есть два типа заголовков новостей:
    # 1. Заголовок размещенный на картинке
    # 2. Заголовок без картинки
    # Поэтому для сайта mail.ru будет два xpath
    'mail': ["//div/span/span[@class='photo__captions']/span[contains(@class, 'photo__title')]",
             "//div/ul/li[@class='list__item']/span/a"],
    'dzen': "//div/ul/li/a/div/span"
}

# индентификаторы пути xpath к источнику новости
xpath_news_source_dict = {
    # для получения источника на сайте Лента.ру нужно перейти по ссылке на саму новость и потом по пути xpath ниже
    # найти источник. К сожалению встречаються новости в которых не смотря на наличие внешнего источника 
    # вытащить данные об источнике крайне сложно так как крайне редко но клас source не используеться
    'lenta': "//div/p/a[@class='source']",
    # источник новости на mail можно найти только после открытия самой страницы новости по следующему xpath
    'mail': "//div[contains(@class, 'breadcrumbs')]/span/span/a/span",
    'dzen': "//div[@class='news-story__head']/a/span[@class='news-story__subtitle-text']"
} 

# идентификатор пути xpath к ссылке на новость
xpath_news_links_dict = {
    'lenta': "//div/ul/li[@class='parts-page__item']/a[@class='card-full-news _parts-news']/@href",
    # На сайте mail.ru есть два типа ссылок на новости:
    # 1. Ссылки размещенные на картинке
    # 2. Ссылки без картинки
    # Поэтому для сайта mail.ru будет два xpath
    'mail': ["//div/span/a[@class='photo__inner link-holder' and contains(@href, 'https://')]/@href",
             "//div/ul/li[@class='list__item']/span/a/@href"],
    'dzen': "//div/ul/li/a/@href"
}

# идентификатор пути xpath к дате новости
xpath_news_date_dict = {
    'lenta': "//div/ul/li/a/div/time[@class='card-full-news__info-item card-full-news__date']",
    # дату новости на mail можно найти только после открытия самой страницы новости по следующему xpath
    'mail': "//div[contains(@class, 'breadcrumbs')]/span/span/span/@datetime",
    # В дзен нет дат новостей поэтому заполним текущей датой
    'dzen': ""
}

In [3]:
# Определим заголовок для выполнения запроса
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
}

### Обработка новостей с сайта lenta.ru

In [4]:
# Выполним запрос к сайту lenta.ru
lenta_response = requests.get(url=news_site_urls_dict['lenta'], headers=headers)

In [5]:
# выполним преобразование полученого ответа из текста в html
lenta_dom = html.fromstring(lenta_response.text)

#### Обработка заголовков новостей lenta.ru

In [6]:
# найдем и сохраним список загаловков новостей
lenta_news_list = lenta_dom.xpath(xpath_news_name_dict['lenta'])

In [7]:
# Выведем список заголовков новостей на lenta.ru
lenta_news_list

['Политолог высказался о возможном содержании речи Байдена в Польше',
 'Молдавия в очередной раз потребовала демилитаризации Приднестровья',
 'Американский военный признался в желании обнять Путина',
 'Бросившего через спину полицейского россиянина отправили под домашний арест',
 'Вероятность заключения многостороннего договора по типу ДСНВ оценили',
 'У дома в российском городе снесло крышу',
 'Крупнейшая авиакомпания Европы отменит тысячи рейсов из-за нехватки сотрудников',
 'В Госдуме прокомментировали приостановку участия России в ДСНВ',
 'Медведев высказался о стратегической стабильности США',
 'В Киеве заявили о получении от Китая пунктов по мирному урегулированию конфликта',
 'Джуд Лоу стал отцом в седьмой раз',
 '«Калашников» рассказал о положительных отзывах об СВЧ',
 'США пригрозили китайским компаниям санкциями за сотрудничество с Россией',
 'Ведущая «Дома-2» раскрыла секрет похудения за две недели после родов',
 'Новый премьер Молдавии дал интервью на фоне карты Молдавского

#### Обработка ссылок на новости lenta.ru

In [8]:
# Получим и сохраним список ссылок на новости с сайта lenta.ru
lenta_links_list = lenta_dom.xpath(xpath_news_links_dict['lenta'])
# Так как для большинства новостей в таге href содержиться только относительный путь к новости
# задаим начальную часть url
lenta_main_link_part = 'https://lenta.ru'

In [9]:
# Выведем список полученых ссылок нв ноыости как есть
lenta_links_list

['/news/2023/02/21/bdnn/',
 '/news/2023/02/21/mld_peacemakers/',
 '/news/2023/02/21/freehugs/',
 '/news/2023/02/21/brosokk/',
 '/news/2023/02/21/novii/',
 '/news/2023/02/21/without_krysha/',
 '/news/2023/02/21/lufthansa/',
 '/news/2023/02/21/komment/',
 '/news/2023/02/21/medvedevzapad/',
 '/news/2023/02/21/kievukrkitai/',
 '/news/2023/02/21/law_dad/',
 '/news/2023/02/21/rifle/',
 '/news/2023/02/21/sancusa/',
 '/news/2023/02/21/orlova/',
 '/news/2023/02/21/velikaya_moldova/',
 '/news/2023/02/21/trapsamoleta/',
 '/news/2023/02/21/students/',
 '/news/2023/02/21/doubtsoncrimea/',
 '/news/2023/02/21/sobyanin_care/',
 '/news/2023/02/21/34995/']

In [10]:
# В списке ссылок встречаються два типа ссылок:
# 1. Относительная ссылка на новость размещенную на сайте lenta.ru
# 2. Полный url в случае если новость размещена на другом сайте. Видимо это дочерний сайт lenta.ru - moslenta.ru
# Преобразуем все относительные ссылки в полный url для первого типа ссылок
for i in range(len(lenta_links_list)):
    if lenta_links_list[i].find('https') < 0:
        lenta_links_list[i] = lenta_main_link_part + lenta_links_list[i]

In [11]:
# Выведем преобразованный список ссылок
lenta_links_list

['https://lenta.ru/news/2023/02/21/bdnn/',
 'https://lenta.ru/news/2023/02/21/mld_peacemakers/',
 'https://lenta.ru/news/2023/02/21/freehugs/',
 'https://lenta.ru/news/2023/02/21/brosokk/',
 'https://lenta.ru/news/2023/02/21/novii/',
 'https://lenta.ru/news/2023/02/21/without_krysha/',
 'https://lenta.ru/news/2023/02/21/lufthansa/',
 'https://lenta.ru/news/2023/02/21/komment/',
 'https://lenta.ru/news/2023/02/21/medvedevzapad/',
 'https://lenta.ru/news/2023/02/21/kievukrkitai/',
 'https://lenta.ru/news/2023/02/21/law_dad/',
 'https://lenta.ru/news/2023/02/21/rifle/',
 'https://lenta.ru/news/2023/02/21/sancusa/',
 'https://lenta.ru/news/2023/02/21/orlova/',
 'https://lenta.ru/news/2023/02/21/velikaya_moldova/',
 'https://lenta.ru/news/2023/02/21/trapsamoleta/',
 'https://lenta.ru/news/2023/02/21/students/',
 'https://lenta.ru/news/2023/02/21/doubtsoncrimea/',
 'https://lenta.ru/news/2023/02/21/sobyanin_care/',
 'https://lenta.ru/news/2023/02/21/34995/']

#### Обработка дат новостей lenta.ru

In [12]:
# Получим список времени для каждой новости
# На сайте lenta.ru указываеться только время новости а не дата
lenta_dates_list = lenta_dom.xpath(xpath_news_date_dict['lenta'])

In [13]:
# Преобразуем список полученых обьектов dom в список времени
for i in range(len(lenta_dates_list)):
    lenta_dates_list[i] = lenta_dates_list[i].text_content()

In [14]:
# Выведем список содержащий время для каждой новости
lenta_dates_list

['16:24',
 '16:23',
 '16:21',
 '16:16',
 '16:15',
 '16:15',
 '16:14',
 '16:14',
 '16:14',
 '16:13',
 '16:12',
 '16:11',
 '16:11',
 '16:09',
 '16:07',
 '16:07',
 '16:04',
 '16:04',
 '15:59',
 '15:59']

In [15]:
# Теперь преобразуем список времени в список datetime
# Полуим текущую дату и преобразуем ее в строку
cur_date = str(date.today())
cur_date

'2023-02-21'

In [16]:
# Выполним преобразование для каждого элемента списка в строковое представление в формате 'дата время'
for i in range(len(lenta_dates_list)):
    lenta_dates_list[i] = cur_date + ' ' + lenta_dates_list[i]

In [17]:
# Выведем финальный список дат новостей
lenta_dates_list

['2023-02-21 16:24',
 '2023-02-21 16:23',
 '2023-02-21 16:21',
 '2023-02-21 16:16',
 '2023-02-21 16:15',
 '2023-02-21 16:15',
 '2023-02-21 16:14',
 '2023-02-21 16:14',
 '2023-02-21 16:14',
 '2023-02-21 16:13',
 '2023-02-21 16:12',
 '2023-02-21 16:11',
 '2023-02-21 16:11',
 '2023-02-21 16:09',
 '2023-02-21 16:07',
 '2023-02-21 16:07',
 '2023-02-21 16:04',
 '2023-02-21 16:04',
 '2023-02-21 15:59',
 '2023-02-21 15:59']

#### Обработка источников новостей lenta.ru

In [18]:
# Для определения источника новости lenta.ru необходимо перейти по ссылке в саму новость 
# и найти в тескте источник, который классифицируеться классом - source
# ссылки на новость возьмем из списка lenta_links_list

# Создадим список источников новостей
lenta_src_list = []

for i in range(len(lenta_links_list)):
    # Для новостей дочернено сайта moslenta.ru будем считать источником данный сайт
    # поэтому обрабатываем ссылки долбко для сайта lenta.ru
    if lenta_links_list[i].find('/lenta.ru') >= 0:
        # Получаем содержание новости
        lenta_news_response = requests.get(url=lenta_links_list[i], headers=headers)
    
        # выполним преобразование полученого ответа из текста в html
        lenta_news_dom = html.fromstring(lenta_news_response.text)
    
        # найдем источник новости
        lenta_news_src = lenta_news_dom.xpath(xpath_news_source_dict['lenta'])
        
        # Если источник есть
        if len(lenta_news_src) > 0:
            # Добавим источник в список
            lenta_src_list.append(lenta_news_src[0].text_content())
        #В противном случае
        else:
            # Добавляем в источник lenta.ru
            lenta_src_list.append('lenta.ru')
    else:
                # найдем наименование сайта источника
        t_src = (lenta_links_list[i].split('/'))[2]
        # и добавим в список
        lenta_src_list.append(t_src)
    

In [19]:
# Выведе
lenta_src_list

['lenta.ru',
 'lenta.ru',
 'lenta.ru',
 'lenta.ru',
 'lenta.ru',
 'lenta.ru',
 'lenta.ru',
 'lenta.ru',
 'ТАСС',
 'РИА Новости',
 'lenta.ru',
 'ТАСС',
 'lenta.ru',
 'lenta.ru',
 'lenta.ru',
 'lenta.ru',
 'lenta.ru',
 'lenta.ru',
 'lenta.ru',
 'РИА Новости']

#### Формируем список словарей новостей lenta.ru

In [20]:
# Создаем пустой список который будет содержать данные по каждой новости найденной на lenta.ru
lenta_news_data_list = []

# Создаем временный словарь который будет содержать данные для одной новости
tmp_news_dict = {
    'news_headline': '',
    'news_date': '',
    'news_link': '',
    'news_source': ''
}

# Наполним список данными
for i in range(len(lenta_news_list)):
    # Вносим заголовок новости
    tmp_news_dict['news_headline'] = lenta_news_list[i]
    # Вносим дату новости
    tmp_news_dict['news_date'] = lenta_dates_list[i]
    # Вносим ссылку на новость
    tmp_news_dict['news_link'] = lenta_links_list[i]
    # Вносим источник новости
    tmp_news_dict['news_source'] = lenta_src_list[i]
    
    # Добавляем словарь в список
    lenta_news_data_list.append(tmp_news_dict.copy())

In [21]:
# Выведем полученый список
lenta_news_data_list

[{'news_headline': 'Политолог высказался о возможном содержании речи Байдена в Польше',
  'news_date': '2023-02-21 16:24',
  'news_link': 'https://lenta.ru/news/2023/02/21/bdnn/',
  'news_source': 'lenta.ru'},
 {'news_headline': 'Молдавия в очередной раз потребовала демилитаризации Приднестровья',
  'news_date': '2023-02-21 16:23',
  'news_link': 'https://lenta.ru/news/2023/02/21/mld_peacemakers/',
  'news_source': 'lenta.ru'},
 {'news_headline': 'Американский военный признался в желании обнять Путина',
  'news_date': '2023-02-21 16:21',
  'news_link': 'https://lenta.ru/news/2023/02/21/freehugs/',
  'news_source': 'lenta.ru'},
 {'news_headline': 'Бросившего через спину полицейского россиянина отправили под домашний арест',
  'news_date': '2023-02-21 16:16',
  'news_link': 'https://lenta.ru/news/2023/02/21/brosokk/',
  'news_source': 'lenta.ru'},
 {'news_headline': 'Вероятность заключения многостороннего договора по типу ДСНВ оценили',
  'news_date': '2023-02-21 16:15',
  'news_link': '

In [22]:
fdate = cur_date + '_' + str((datetime.now()).hour) + '-' + str((datetime.now()).minute)

In [23]:
fdate

'2023-02-21_16-29'

In [24]:
# Сохраним полученный список новостей в json формате
with open('lenta_news_' + fdate + '.json', 'w') as outfile:
    json.dump(lenta_news_data_list, outfile)

In [25]:
# Теперь проверим сохраненные данные
with open('lenta_news_' + fdate + '.json') as f_json:
    lenta_news_json = json.load(f_json)

In [26]:
lenta_news_json

[{'news_headline': 'Политолог высказался о возможном содержании речи Байдена в Польше',
  'news_date': '2023-02-21 16:24',
  'news_link': 'https://lenta.ru/news/2023/02/21/bdnn/',
  'news_source': 'lenta.ru'},
 {'news_headline': 'Молдавия в очередной раз потребовала демилитаризации Приднестровья',
  'news_date': '2023-02-21 16:23',
  'news_link': 'https://lenta.ru/news/2023/02/21/mld_peacemakers/',
  'news_source': 'lenta.ru'},
 {'news_headline': 'Американский военный признался в желании обнять Путина',
  'news_date': '2023-02-21 16:21',
  'news_link': 'https://lenta.ru/news/2023/02/21/freehugs/',
  'news_source': 'lenta.ru'},
 {'news_headline': 'Бросившего через спину полицейского россиянина отправили под домашний арест',
  'news_date': '2023-02-21 16:16',
  'news_link': 'https://lenta.ru/news/2023/02/21/brosokk/',
  'news_source': 'lenta.ru'},
 {'news_headline': 'Вероятность заключения многостороннего договора по типу ДСНВ оценили',
  'news_date': '2023-02-21 16:15',
  'news_link': '

### Обработка новостей с сайта mail.ru

In [27]:
# Выполним запрос к сайту mail.ru
mail_response = requests.get(url=news_site_urls_dict['mail'], headers=headers)

In [28]:
# выполним преобразование полученого ответа из текста в html
mail_dom = html.fromstring(mail_response.text)

#### Обработка заголовков новостей mail.ru

In [29]:
# найдем и сохраним список загаловков новостей с заголовками на картинках
mail_news_list_pic = mail_dom.xpath(xpath_news_name_dict['mail'][0])
# найдем и сохраним список загаловков новостей без картинкок
mail_news_list_no_pic = mail_dom.xpath(xpath_news_name_dict['mail'][1])

In [30]:
# Преобразуем список обьектов dom в список заголовков для заголовков с картинками
for i in range(len(mail_news_list_pic)):
    # заодно удалим служебные символы из заголовков
    mail_news_list_pic[i] = (mail_news_list_pic[i].text_content()).replace('\xa0', ' ')

In [31]:
# Выведем полученый список заголовков
mail_news_list_pic

['Послание Путина Федеральному собранию. Главное',
 'Россия приостанавливает участие в договоре об СНВ',
 'Столтенберг заявил о разрушении системы ядерного сдерживания',
 'Бортников: РФ не давала гарантий Байдену во время визита в Киев',
 'С 1 марта услуги ЖКХ начнут оплачивать по новым правилам']

In [32]:
# Преобразуем список обьектов dom в список заголовков для заголовков без картинок
for i in range(len(mail_news_list_no_pic)):
    # заодно удалим служебные символы из заголовков
    mail_news_list_no_pic[i] = (mail_news_list_no_pic[i].text_content()).replace('\xa0', ' ')

In [33]:
# Выведем полученый список заголовков
mail_news_list_no_pic

['СМИ: в Киеве опасаются, что США принудят Украину к сделке с РФ',
 'В Польше заявили о крахе рынка яблок из-за санкций против России',
 'Неизвестный шар диаметром 1,5 метра выбросило на пляж в Японии',
 'Премьер Италии прибыла в Киев для встречи с Зеленским',
 'Тест: Знаете ли вы значение сложных слов',
 'Российские участницы турнира в Иране получат хиджабы в аэропорту',
 'Путин перечислил меры поддержки жителей новых регионов',
 '«Никаких выводов не сделано». Стрелков раскритиковал содержание послания Путина',
 'Владимир Путин объявил о реформе высшего образования',
 'Силуанов отверг возможность повышения налогов',
 'В России увеличат размер страховки для добровольных пенсионных накоплений',
 'Путин предложил запустить программу льготной аренды жилья для работников ОПК',
 'Исследование: поглаживания по руке снижают стресс и тревогу',
 'Ученые обнаружили неизвестный класс богатых водой астероидов',
 'Путин объявил о создании фонда поддержки ветеранов',
 'AFP: украинские военные провод

In [34]:
#Обьеденим списки заголовков в один
mail_news_list = mail_news_list_pic + mail_news_list_no_pic

In [35]:
# Выведем обьедененный список
mail_news_list

['Послание Путина Федеральному собранию. Главное',
 'Россия приостанавливает участие в договоре об СНВ',
 'Столтенберг заявил о разрушении системы ядерного сдерживания',
 'Бортников: РФ не давала гарантий Байдену во время визита в Киев',
 'С 1 марта услуги ЖКХ начнут оплачивать по новым правилам',
 'СМИ: в Киеве опасаются, что США принудят Украину к сделке с РФ',
 'В Польше заявили о крахе рынка яблок из-за санкций против России',
 'Неизвестный шар диаметром 1,5 метра выбросило на пляж в Японии',
 'Премьер Италии прибыла в Киев для встречи с Зеленским',
 'Тест: Знаете ли вы значение сложных слов',
 'Российские участницы турнира в Иране получат хиджабы в аэропорту',
 'Путин перечислил меры поддержки жителей новых регионов',
 '«Никаких выводов не сделано». Стрелков раскритиковал содержание послания Путина',
 'Владимир Путин объявил о реформе высшего образования',
 'Силуанов отверг возможность повышения налогов',
 'В России увеличат размер страховки для добровольных пенсионных накоплений'

#### Обработка ссылок новостей mail.ru

In [36]:
# найдем и сохраним список ссылок на новости с заголовками на картинках
mail_links_list_pic = mail_dom.xpath(xpath_news_links_dict['mail'][0])
# найдем и сохраним список загаловков новостей без картинкок
mail_links_list_no_pic = mail_dom.xpath(xpath_news_links_dict['mail'][1])

In [37]:
# Выведем полученый список ссылок
mail_links_list_pic

['https://news.mail.ru/politics/55150018/',
 'https://news.mail.ru/politics/55149683/',
 'https://news.mail.ru/politics/55151395/',
 'https://news.mail.ru/politics/55151613/',
 'https://news.mail.ru/economics/55141064/']

In [38]:
# Выведем полученый список ссылок
mail_links_list_no_pic

['https://news.mail.ru/politics/55144307/',
 'https://news.mail.ru/economics/55141779/',
 'https://news.mail.ru/society/55143955/',
 'https://news.mail.ru/politics/55148410/',
 'https://news.mail.ru/society/55147867/',
 'https://sportmail.ru/news/weight-lifting/55145867/',
 'https://news.mail.ru/politics/55149302/',
 'https://news.mail.ru/politics/55150416/',
 'https://news.mail.ru/politics/55149346/',
 'https://news.mail.ru/economics/55151311/',
 'https://news.mail.ru/economics/55148612/',
 'https://news.mail.ru/economics/55148081/',
 'https://news.mail.ru/society/55151344/',
 'https://pogoda.mail.ru/news/55150396/',
 'https://news.mail.ru/society/55148061/',
 'https://news.mail.ru/incident/55143207/',
 'https://news.mail.ru/incident/55143745/',
 'https://news.mail.ru/incident/55140411/']

In [39]:
#Обьеденим списки ссылок в один
mail_links_list = mail_links_list_pic + mail_links_list_no_pic

In [40]:
# Выведем обьедененный список
mail_links_list

['https://news.mail.ru/politics/55150018/',
 'https://news.mail.ru/politics/55149683/',
 'https://news.mail.ru/politics/55151395/',
 'https://news.mail.ru/politics/55151613/',
 'https://news.mail.ru/economics/55141064/',
 'https://news.mail.ru/politics/55144307/',
 'https://news.mail.ru/economics/55141779/',
 'https://news.mail.ru/society/55143955/',
 'https://news.mail.ru/politics/55148410/',
 'https://news.mail.ru/society/55147867/',
 'https://sportmail.ru/news/weight-lifting/55145867/',
 'https://news.mail.ru/politics/55149302/',
 'https://news.mail.ru/politics/55150416/',
 'https://news.mail.ru/politics/55149346/',
 'https://news.mail.ru/economics/55151311/',
 'https://news.mail.ru/economics/55148612/',
 'https://news.mail.ru/economics/55148081/',
 'https://news.mail.ru/society/55151344/',
 'https://pogoda.mail.ru/news/55150396/',
 'https://news.mail.ru/society/55148061/',
 'https://news.mail.ru/incident/55143207/',
 'https://news.mail.ru/incident/55143745/',
 'https://news.mail.ru

#### Обработка дат новостей mail.ru

In [41]:
# Для обработки дат новостей сайта mail.ru необходимо зайти по ссылки на каждую новость
# после чего найти дату новости 

In [42]:
# Обработаем даты для новостей 
# Создадим список дат новостей
mail_dates_list = []

for i in range(len(mail_links_list)):
    # Выполним запрос новости mail.ru
    mail_one_news_response = requests.get(url=mail_links_list[i], headers=headers)
    # выполним преобразование полученого ответа из текста в html
    mail_one_news_dom = html.fromstring(mail_one_news_response.text)
    
    # Найдем дату новости
    mail_one_date = mail_one_news_dom.xpath(xpath_news_date_dict['mail'])
    # Преобразуем дату к более читаемому формату
    t_date = mail_one_date[0].split('T')
    t_time = t_date[1].split(':')
    mail_one_date = t_date[0] + ' ' + t_time[0] + ':' + t_time[1] 
    
    # Запишим дату в список дат новостей
    mail_dates_list.append(mail_one_date)

In [43]:
# Выведем список дат новостей
mail_dates_list

['2023-02-21 14:16',
 '2023-02-21 13:50',
 '2023-02-21 16:03',
 '2023-02-21 16:09',
 '2023-02-21 06:53',
 '2023-02-21 09:54',
 '2023-02-21 07:37',
 '2023-02-21 10:16',
 '2023-02-21 13:03',
 '2023-02-21 12:56',
 '2023-02-21 11:38',
 '2023-02-21 14:56',
 '2023-02-21 14:29',
 '2023-02-21 13:36',
 '2023-02-21 15:16',
 '2023-02-21 13:26',
 '2023-02-21 13:07',
 '2023-02-21 15:38',
 '2023-02-21 14:41',
 '2023-02-21 13:01',
 '2023-02-21 10:50',
 '2023-02-21 09:08',
 '2023-02-20 22:09']

#### Обработка источников новостей mail.ru

In [44]:
# Для обработки источников новостей сайта mail.ru необходимо зайти по ссылки на каждую новость
# после чего найти источник новости 

In [45]:
# Обработаем источники новостей

# Создадим список дат новостей
mail_src_list = []

for i in range(len(mail_links_list)):
    # Выполним запрос новости mail.ru
    mail_one_src_response = requests.get(url=mail_links_list[i], headers=headers)
    # выполним преобразование полученого ответа из текста в html
    mail_one_src_dom = html.fromstring(mail_one_src_response.text)
    
    # Найдем источник новости
    mail_one_src = mail_one_src_dom.xpath(xpath_news_source_dict['mail'])
    
    # Запишим дату в список дат новостей
    mail_src_list.append(mail_one_src[0].text_content())

In [46]:
# Выведем список источников новостей
mail_src_list

['Коммерсантъ',
 'Ведомости',
 'РБК',
 'Коммерсантъ',
 'Известия',
 'ТАСС',
 'Lenta.Ru',
 'Life.ru',
 '© РИА Новости',
 'Новости Mail.ru',
 'ТАСС',
 '© РИА Новости',
 'RTVi',
 'Ведомости',
 'Ведомости',
 '© РИА Новости',
 'ТАСС',
 'Новости Mail.ru',
 'Погода Mail.ru',
 'Ведомости',
 'Коммерсантъ',
 'РБК',
 'ТАСС']

#### Формируем список словарей новостей mail.ru

In [47]:
# Создаем пустой список который будет содержать данные по каждой новости найденной на mail.ru
mail_news_data_list = []

# Создаем временный словарь который будет содержать данные для одной новости
tmp_news_dict = {
    'news_headline': '',
    'news_date': '',
    'news_link': '',
    'news_source': ''
}

# Наполним список данными
for i in range(len(mail_news_list)):
    # Вносим заголовок новости
    tmp_news_dict['news_headline'] = mail_news_list[i]
    # Вносим дату новости
    tmp_news_dict['news_date'] = mail_dates_list[i]
    # Вносим ссылку на новость
    tmp_news_dict['news_link'] = mail_links_list[i]
    # Вносим источник новости
    tmp_news_dict['news_source'] = mail_src_list[i]
    
    # Добавляем словарь в список
    mail_news_data_list.append(tmp_news_dict.copy())

In [48]:
# Выведем полученый список
mail_news_data_list

[{'news_headline': 'Послание Путина Федеральному собранию. Главное',
  'news_date': '2023-02-21 14:16',
  'news_link': 'https://news.mail.ru/politics/55150018/',
  'news_source': 'Коммерсантъ'},
 {'news_headline': 'Россия приостанавливает участие в договоре об СНВ',
  'news_date': '2023-02-21 13:50',
  'news_link': 'https://news.mail.ru/politics/55149683/',
  'news_source': 'Ведомости'},
 {'news_headline': 'Столтенберг заявил о разрушении системы ядерного сдерживания',
  'news_date': '2023-02-21 16:03',
  'news_link': 'https://news.mail.ru/politics/55151395/',
  'news_source': 'РБК'},
 {'news_headline': 'Бортников: РФ не давала гарантий Байдену во время визита в Киев',
  'news_date': '2023-02-21 16:09',
  'news_link': 'https://news.mail.ru/politics/55151613/',
  'news_source': 'Коммерсантъ'},
 {'news_headline': 'С 1 марта услуги ЖКХ начнут оплачивать по новым правилам',
  'news_date': '2023-02-21 06:53',
  'news_link': 'https://news.mail.ru/economics/55141064/',
  'news_source': 'Извес

In [49]:
fdate = cur_date + '_' + str((datetime.now()).hour) + '-' + str((datetime.now()).minute)

In [50]:
fdate

'2023-02-21_16-29'

In [51]:
# Сохраним полученный список новостей в json формате
with open('mail_news_' + fdate + '.json', 'w') as outfile:
    json.dump(mail_news_data_list, outfile)

In [52]:
# Теперь проверим сохраненные данные
with open('mail_news_' + fdate + '.json') as f_json:
    mail_news_json = json.load(f_json)

In [53]:
mail_news_json

[{'news_headline': 'Послание Путина Федеральному собранию. Главное',
  'news_date': '2023-02-21 14:16',
  'news_link': 'https://news.mail.ru/politics/55150018/',
  'news_source': 'Коммерсантъ'},
 {'news_headline': 'Россия приостанавливает участие в договоре об СНВ',
  'news_date': '2023-02-21 13:50',
  'news_link': 'https://news.mail.ru/politics/55149683/',
  'news_source': 'Ведомости'},
 {'news_headline': 'Столтенберг заявил о разрушении системы ядерного сдерживания',
  'news_date': '2023-02-21 16:03',
  'news_link': 'https://news.mail.ru/politics/55151395/',
  'news_source': 'РБК'},
 {'news_headline': 'Бортников: РФ не давала гарантий Байдену во время визита в Киев',
  'news_date': '2023-02-21 16:09',
  'news_link': 'https://news.mail.ru/politics/55151613/',
  'news_source': 'Коммерсантъ'},
 {'news_headline': 'С 1 марта услуги ЖКХ начнут оплачивать по новым правилам',
  'news_date': '2023-02-21 06:53',
  'news_link': 'https://news.mail.ru/economics/55141064/',
  'news_source': 'Извес

### Обработка новостей с сайта dzen.ru (yandex)

In [54]:
# Выполним запрос к сайту dzen.ru
# Для dzen как и в целом для яндекс нужно вызывать метод post
dzen_response = requests.post(url=news_site_urls_dict['dzen'], headers=headers)

In [55]:
# выполним преобразование полученого ответа из текста в html
dzen_dom = html.fromstring(dzen_response.text)

#### Обработка заголовков новостей mail.ru

In [56]:
# найдем и сохраним список загаловков новостей на dzen
dzen_news_list = dzen_dom.xpath(xpath_news_name_dict['dzen'])

In [57]:
# Преобразуем список обьектов dom в список заголовков
for i in range(len(dzen_news_list)):
    # заодно удалим служебные символы из заголовков
    dzen_news_list[i] = (dzen_news_list[i].text_content()).replace('\xa0', ' ').strip()

In [58]:
# Выведем полученый список заголовков
dzen_news_list

['Путин объявил о приостановке участия РФ в договоре об СНВ-3',
 'МИД: Послу США заявлен демарш из-за роста вовлеченности в боевые действия на Украине',
 'В МЧС России подтвердили гибель одного из членов экипажа затонувшего сухогруза Seamark',
 'Путин предложил вернуться к традиционному сроку обучения в вузах от 4 до 6 лет',
 'Путин предложил с 2024 года повысить МРОТ до 19 242 рублей',
 'Трасса, соединяющая Калужское и Варшавское шоссе, откроется в ТиНАО осенью',
 'На Цветном бульваре открылась посвященная Большой кольцевой линии выставка',
 'Началось строительство крытого перехода из метро «Прокшино» в одноименный бизнес-квартал',
 'Загрутдинов: Дом по программе реновации введут в Новогирееве',
 'На фестивале «Московская Масленица» соберут подарки для участников СВО',
 'Владимир Путин сообщил о приостановке участия России в договоре СНВ-3',
 'В Китае в древнем Юэяне нашли туалет со смывом, созданный 2400 лет назад',
 '«Ъ»: китайские грузовики заняли более половины рынка России в янва

#### Обработка ссылок новостей dzen.ru

In [59]:
# найдем и сохраним список ссылок на новости
dzen_links_list = dzen_dom.xpath(xpath_news_links_dict['dzen'])

In [60]:
# Выведем полученый список ссылок
dzen_links_list

['https://dzen.ru/news/story/Putin_obyavil_opriostanovke_uchastiya_RF_vdogovore_ob_SNV-3--3426cb0ae7e46c8d47172076286daf15?lang=ru&from=main_portal&fan=1&stid=VlUydyzoaIn8b2FOg8Lm&t=1676985665&persistent_id=2070960327&story=5b3e1db8-b611-50df-a6a9-73e7121d06dc&issue_tld=ru&utm_referrer=dzen.ru',
 'https://dzen.ru/news/story/MID_Poslu_SSHA_zayavlen_demarsh_iz-zarosta_vovlechennosti_vboevye_dejstviya_naUkraine--2cd8fbc05cb3ba08a97d8bee9229f845?lang=ru&from=main_portal&fan=1&stid=pnwm4BU2Tcjzw9pQ0GYn&t=1676985665&persistent_id=2072160417&story=c0c0bb4c-b393-5988-a9e1-b44ea9c0474e&issue_tld=ru&utm_referrer=dzen.ru',
 'https://dzen.ru/news/story/VMCHS_Rossii_podtverdili_gibel_odnogo_izchlenov_ehkipazha_zatonuvshego_sukhogruza_Seamark--340a52abeecdae97e5c627b5de420804?lang=ru&from=main_portal&fan=1&stid=g1Vs_dm_-V7RW6UpsO8E&t=1676985665&persistent_id=2076560069&story=0c1ded21-d319-5cc7-af24-3df3df3ccc39&issue_tld=ru&utm_referrer=dzen.ru',
 'https://dzen.ru/news/story/Putin_predlozhil_vernuts

#### Обработка дат новостей dzen.ru

In [61]:
# Я не нашел даты для новостей так как dzen их не указывает поэтому принимаем за дату новости дату парсинга

In [62]:
# Обработаем даты для новостей 
#Сформируем дату парсинга новостей
dzen_news_date = str(date.today()) + ' 12:00'

#### Обработка источников новостей dzen.ru

In [63]:
# Для обработки источников новостей сайта dzen.ru необходимо зайти по ссылки на каждую новость
# после чего найти источник новости 

In [64]:
# Обработаем источники новостей

# Создадим список дат новостей
dzen_src_list = []

for i in range(len(dzen_links_list)):
    # Выполним запрос новости dzen.ru
    dzen_one_src_response = requests.post(url=dzen_links_list[i], headers=headers)
    # выполним преобразование полученого ответа из текста в html
    dzen_one_src_dom = html.fromstring(dzen_one_src_response.text)
    
    # Найдем источник новости
    dzen_one_src = dzen_one_src_dom.xpath(xpath_news_source_dict['dzen'])
    
    # Запишим дату в список дат новостей
    dzen_src_list.append(dzen_one_src[0].text_content())

In [65]:
# Выведем список источников новостей
dzen_src_list

['Интерфакс',
 'РБК',
 'RT на русском',
 'Российская газета',
 'Коммерсантъ',
 'РИАМО',
 'Известия',
 'Москва 24',
 'Информационный центр Правительства Москвы',
 'Официальный портал Мэра и Правительства Москвы',
 'РИА Новости',
 'РИА Новости',
 'Автоновости дня',
 'Общественная служба новостей',
 'Российская газета',
 'РИА Новости',
 'Lenta.ru',
 'РИА Новости',
 'Известия',
 'Lenta.ru',
 'Коммерсантъ',
 'Российская газета',
 'Lenta.ru',
 'Газета.Ru',
 'ТАСС',
 'BFM.ru',
 'РБК',
 'Коммерсантъ',
 'ТАСС',
 'Газета.Ru',
 'Известия',
 'РИА Новости',
 'REGNUM',
 'ТАСС',
 'Газета.Ru',
 'RT на русском',
 'CyberSport.ru',
 'Матч ТВ',
 'Игромания.ру',
 'Чемпионат',
 'Lenta.ru',
 'REGNUM',
 'Российская газета',
 'Матч ТВ',
 'РБК',
 'Канобу',
 'РИА “Informing”',
 'РБК',
 'Topdaynews.ru',
 'Газета Культура',
 'Пепелац Ньюс',
 'Игромания.ру',
 'Игромания.ру',
 'Superplanshet.ru',
 'Anti-Malware.ru',
 'РИА Новости',
 'Юность Сибири',
 'Solenka.info',
 'Solenka.info',
 'Санкт-Петербург.ру',
 'Автоново

#### Формируем список словарей новостей dzen.ru

In [66]:
# Создаем пустой список который будет содержать данные по каждой новости найденной на dzen.ru
dzen_news_data_list = []

# Создаем временный словарь который будет содержать данные для одной новости
tmp_news_dict = {
    'news_headline': '',
    'news_date': '',
    'news_link': '',
    'news_source': ''
}

# Наполним список данными
for i in range(len(dzen_news_list)):
    # Вносим заголовок новости
    tmp_news_dict['news_headline'] = dzen_news_list[i]
    # Вносим дату новости
    tmp_news_dict['news_date'] = dzen_news_date
    # Вносим ссылку на новость
    tmp_news_dict['news_link'] = dzen_links_list[i]
    # Вносим источник новости
    tmp_news_dict['news_source'] = dzen_src_list[i]
    
    # Добавляем словарь в список
    dzen_news_data_list.append(tmp_news_dict.copy())

In [67]:
# Выведем полученый список
dzen_news_data_list

[{'news_headline': 'Путин объявил о приостановке участия РФ в договоре об СНВ-3',
  'news_date': '2023-02-21 12:00',
  'news_link': 'https://dzen.ru/news/story/Putin_obyavil_opriostanovke_uchastiya_RF_vdogovore_ob_SNV-3--3426cb0ae7e46c8d47172076286daf15?lang=ru&from=main_portal&fan=1&stid=VlUydyzoaIn8b2FOg8Lm&t=1676985665&persistent_id=2070960327&story=5b3e1db8-b611-50df-a6a9-73e7121d06dc&issue_tld=ru&utm_referrer=dzen.ru',
  'news_source': 'Интерфакс'},
 {'news_headline': 'МИД: Послу США заявлен демарш из-за роста вовлеченности в боевые действия на Украине',
  'news_date': '2023-02-21 12:00',
  'news_link': 'https://dzen.ru/news/story/MID_Poslu_SSHA_zayavlen_demarsh_iz-zarosta_vovlechennosti_vboevye_dejstviya_naUkraine--2cd8fbc05cb3ba08a97d8bee9229f845?lang=ru&from=main_portal&fan=1&stid=pnwm4BU2Tcjzw9pQ0GYn&t=1676985665&persistent_id=2072160417&story=c0c0bb4c-b393-5988-a9e1-b44ea9c0474e&issue_tld=ru&utm_referrer=dzen.ru',
  'news_source': 'РБК'},
 {'news_headline': 'В МЧС России подт

In [68]:
fdate = cur_date + '_' + str((datetime.now()).hour) + '-' + str((datetime.now()).minute)

In [69]:
fdate

'2023-02-21_16-30'

In [70]:
# Сохраним полученный список новостей в json формате
with open('dzen_news_' + fdate + '.json', 'w') as outfile:
    json.dump(dzen_news_data_list, outfile)

In [71]:
# Теперь проверим сохраненные данные
with open('dzen_news_' + fdate + '.json') as f_json:
    dzen_news_json = json.load(f_json)

In [72]:
dzen_news_json

[{'news_headline': 'Путин объявил о приостановке участия РФ в договоре об СНВ-3',
  'news_date': '2023-02-21 12:00',
  'news_link': 'https://dzen.ru/news/story/Putin_obyavil_opriostanovke_uchastiya_RF_vdogovore_ob_SNV-3--3426cb0ae7e46c8d47172076286daf15?lang=ru&from=main_portal&fan=1&stid=VlUydyzoaIn8b2FOg8Lm&t=1676985665&persistent_id=2070960327&story=5b3e1db8-b611-50df-a6a9-73e7121d06dc&issue_tld=ru&utm_referrer=dzen.ru',
  'news_source': 'Интерфакс'},
 {'news_headline': 'МИД: Послу США заявлен демарш из-за роста вовлеченности в боевые действия на Украине',
  'news_date': '2023-02-21 12:00',
  'news_link': 'https://dzen.ru/news/story/MID_Poslu_SSHA_zayavlen_demarsh_iz-zarosta_vovlechennosti_vboevye_dejstviya_naUkraine--2cd8fbc05cb3ba08a97d8bee9229f845?lang=ru&from=main_portal&fan=1&stid=pnwm4BU2Tcjzw9pQ0GYn&t=1676985665&persistent_id=2072160417&story=c0c0bb4c-b393-5988-a9e1-b44ea9c0474e&issue_tld=ru&utm_referrer=dzen.ru',
  'news_source': 'РБК'},
 {'news_headline': 'В МЧС России подт

In [73]:
dzen_response.ok


True

In [74]:
dzen_response.status_code

200